In [74]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
SparkSession.builder.config(conf=SparkConf())

import logging

from elasticsearch import Elasticsearch
import pandas as pd

# Enable Arrow-based columnar data transfers
#SparkConf.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

es_header = [{
'host': 'https://hn-search-9858436033.us-east-1.bonsaisearch.net',
'port': 443,
'use_ssl': True,
'http_auth': ("i1gd3mbo4r","sf8q52fulp")
}]

client = Elasticsearch(
        ['https://hn-search-9858436033.us-east-1.bonsaisearch.net:443'],
        http_auth=("t2mABiEPJn","KwyXmSUap5AMfcR34Lvi"))


def create_spark_configuration():
    spark_config = None

    try:
        spark_config = (SparkSession.builder
            .appName("ElasticsearchSparkIntegration")
            .config("spark.jars.packages", "org.elasticsearch:elasticsearch-spark-20_2.12:7.13.4")#7.17.14
            .config("spark.sql.execution.arrow.pyspark.enabled", "true")
            .getOrCreate())
        
        logging.info("Spark connection created successfully!")
    except Exception as e:
        logging.error(f"Couldn't create the spark session due to exception {e}")

    return spark_config

spark = create_spark_configuration()

In [71]:
def openFile(file_path, mode='r'):
    try:
        return open(file_path, mode)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise

def getDataframeFromFile(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('|')
        
        item = {}

        if len(rec) > 6:
            item['movieId'] = rec[0]
            item['name'] = rec[1]
            item['release_date'] = rec[2]
            item['url'] = rec[4]

            #5 => 23
            item['genre'] = [genre[idx]['name'] for  idx, g in enumerate(range(5,24)) if rec[g] == '1']

        elif len(rec) == 2:
            item['id'] = rec[1]
            item['name'] = rec[0]
            
        elif len(rec) == 5:
            item['userId'] = rec[0]
            item['age'] = rec[1]
            item['gender'] = rec[2]
            item['function'] = rec[3]
        else:
            print("Invalid record format. Skipping.")
            return

        data.append(item)
    filed.close()
    return data

def getDataframe(filed):
    data = []
    while True:
        line = filed.readline()
        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('\t')

        item = {}
        item['userId'] = rec[0]
        item['movieId'] = rec[1]
        item['rating'] = rec[2]
        item['timestamp'] = rec[3]
        data.append(item)
    filed.close()
    return data

#=>Data
fileGenre = openFile('../data/u.genre')
fileItem = openFile('../data/u.item')
fileUser = openFile('../data/u.user')
fileData = openFile('../data/u.data')

df_relationship_data = pd.DataFrame(getDataframe(fileData))

genre = getDataframeFromFile(fileGenre)
items = getDataframeFromFile(fileItem)
users = getDataframeFromFile(fileUser)

df_movies = pd.DataFrame(items)
df_users = pd.DataFrame(users)

In [76]:
sdf_users = spark.createDataFrame(df_users)

def create_selection_df_from_users_df(spark_df):
    schema = StructType([
        StructField("id", IntegerType(), False),
        StructField("adult", BooleanType(), False),
        StructField("backdrop_path", StringType(), False),
        StructField("genre_ids", ArrayType(IntegerType()), False),
        StructField("original_language", StringType(), False),
        StructField("original_title", StringType(), False),
        StructField("overview", StringType(), False),
        StructField("popularity", DoubleType(), False),
        StructField("poster_path", StringType(), False),
        StructField("release_date", StringType(), False),
        StructField("title", StringType(), False),
        StructField("video", BooleanType(), False),
        StructField("vote_average", DoubleType(), False),
        StructField("vote_count", IntegerType(), False)
    ])

    sel = (spark_df.selectExpr("CAST(value AS STRING)") \
        .select(from_json(col('value'), schema).alias('data')).select("data.*") \
        .withColumn("description", concat(col('title'),lit(' : '),col('overview'))) \
        .withColumn('popularity',col("popularity").cast('double')) \
        .withColumn('vote_average',col("vote_average").cast('double')) \
        .withColumn('release_date',to_date(col('release_date'),"yyyy-MM-dd")) \
        .select('id','adult','genre_ids','original_language','original_title','description','popularity','release_date','video','vote_average','vote_count','backdrop_path','poster_path'))

    return sel

C:\BigDataSetup\Spark\spark-3.2.4-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:329: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
C:\BigDataSetup\Spark\spark-3.2.4-bin-hadoop2.7\python\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


DataFrame[userId: string, age: string, gender: string, function: string]

In [69]:
# for user in df_users.itertuples():
#     client.index(index='users-index',body={
#         'userId':user.userId,
#         'age':user.age,
#         'gender':user.gender,
#         'function':user.function
#     },id=user.userId)